<a href="https://colab.research.google.com/github/dsynderg/CS-479-machine-translation/blob/main/multilingual_with_scentance_peice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentencepiece

In [ ]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(
  input='src_train.txt',
  model_prefix='english_sp_model',
  vocab_size=16000,
)

spm.SentencePieceTrainer.train(
  input='tgt_train.txt',
  model_prefix='spanish_sp_model',
  vocab_size=16000,
)

In [ ]:
!pip install OpenNMT-py
!pip install "numpy<2.0" # This fixes an error caused by OpenNMT not being maintained

In [2]:
# Build YAML Config file
yaml_text = """
# sentance_peice_yaml.yaml

## Where the samples will be written
save_data: /content/example

src_vocab: /content/example.vocab.src
tgt_vocab: /content/example.vocab.tgt

# Prevent overwriting existing files in the folder
overwrite: True

# -- Sentencepiece Params --
# Tokenization options
src_subword_type: sentencepiece
src_subword_model: english_sp_model.model
tgt_subword_type: sentencepiece
tgt_subword_model: spanish_sp_model.model

# Number of candidates for SentencePiece sampling
subword_nbest: 20
# Smoothing parameter for SentencePiece sampling
subword_alpha: 0.1
# Specific arguments for pyonmttok
src_onmttok_kwargs: "{'mode': 'none', 'spacer_annotate': True}"
tgt_onmttok_kwargs: "{'mode': 'none', 'spacer_annotate': True}"


# Corpus opts:
data:
    corpus_1:
        path_src: /content/src_train.txt
        path_tgt: /content/tgt_train.txt
        transforms: [onmt_tokenize]
    valid:
        path_src: /content/src_val.txt
        path_tgt: /content/tgt_val.txt
        transforms: [onmt_tokenize]

# Train on a single GPU
bucket_size: 10000
world_size: 1
gpu_ranks: [0]

# Where to save the checkpoints
save_model: /content/modle
save_checkpoint_steps: 500
train_steps: 20000
valid_steps: 500
"""

with open("sentance_peice_yaml.yaml", "w") as f:
    f.write(yaml_text)

In [ ]:
!onmt_build_vocab -config sentance_peice_yaml.yaml

In [ ]:
!onmt_train -config sentance_peice_yaml.yaml

In [ ]:
translate_yaml_text = """
model: /content/modle_step_20000_sentance_peice.pt
src: /content/src_test.txt
output: /content/sentancepeice_2000.txt
gpu: 0
verbose: True

# Sentencepiece Params

# Tokenization options
transforms: onmt_tokenize
src_subword_type: sentencepiece
src_subword_model: english_sp_model.model
tgt_subword_type: sentencepiece
tgt_subword_model: spanish_sp_model.model

src_subword_nbest: 20
src_subword_alpha: 0.1
"""

with open("translate_yaml.yaml", "w") as f:
    f.write(translate_yaml_text)

In [ ]:
!onmt_translate -config translate_yaml.yaml